In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

In [2]:
dataframe = pd.read_csv("train.csv")
dataframe.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [3]:
# Create new columns based on the Gender column
dataframe['Is_Male'] = dataframe['Gender'].apply(lambda x: 1 if x == 'Male' else 0)
dataframe['Is_Female'] = dataframe['Gender'].apply(lambda x: 1 if x == 'Female' else 0)

# Create new columns based on the Geography column
dataframe['Is_Germany'] = dataframe['Geography'].apply(lambda x: 1 if x == 'Germany' else 0)
dataframe['Is_Spain'] = dataframe['Geography'].apply(lambda x: 1 if x == 'Spain' else 0)
dataframe['Is_France'] = dataframe['Geography'].apply(lambda x: 1 if x == 'France' else 0)

In [4]:
# drop non-int columns
dataframe.drop(['Geography', 'Gender'], axis = 1, inplace = True)

In [5]:
y = dataframe.Exited.values
X = dataframe.drop(columns=['id','CustomerId','Surname','Exited'])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [6]:
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=13))
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customise metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")



Epoch 1/100


3868/3868 [==============================] - 3s 707us/step - loss: 0.3558 - accuracy: 0.8497
Epoch 2/100
3868/3868 [==============================] - 3s 781us/step - loss: 0.3309 - accuracy: 0.8612
Epoch 3/100
3868/3868 [==============================] - 3s 729us/step - loss: 0.3273 - accuracy: 0.8631
Epoch 4/100
3868/3868 [==============================] - 3s 706us/step - loss: 0.3258 - accuracy: 0.8636
Epoch 5/100
3868/3868 [==============================] - 3s 706us/step - loss: 0.3253 - accuracy: 0.8639
Epoch 6/100
3868/3868 [==============================] - 3s 707us/step - loss: 0.3246 - accuracy: 0.8645
Epoch 7/100
3868/3868 [==============================] - 3s 721us/step - loss: 0.3242 - accuracy: 0.8646
Epoch 8/100
3868/3868 [==============================] - 3s 720us/step - loss: 0.3237 - accuracy: 0.8645
Epoch 9/100
3868/3868 [==============================] - 3s 707us/step - loss: 0.3236 - accuracy: 0.8643
Epoch 10/100
3868/3868 [===========================

In [8]:
X_scaled = X_scaler.transform(X)

predictions = nn_model.predict(X_scaled)

predicted_labels = (predictions > 0.5).astype(int)

5158/5158 [==============================] - 3s 560us/step


In [9]:
predicted_df = pd.DataFrame(predicted_labels)

In [10]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc

conf_matrix = confusion_matrix(y, predicted_labels)

class_report = classification_report(y, predicted_labels)

accuracy = accuracy_score(y, predicted_labels)

print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)
print("\nAccuracy Score:", accuracy)

Confusion Matrix:
 [[123769   6344]
 [ 15770  19151]]

Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.95      0.92    130113
           1       0.75      0.55      0.63     34921

    accuracy                           0.87    165034
   macro avg       0.82      0.75      0.78    165034
weighted avg       0.86      0.87      0.86    165034


Accuracy Score: 0.866003369002751


In [12]:
import joblib

filename = 'nn_model'
joblib.dump(nn_model, filename)

['nn_model']